<a href="https://colab.research.google.com/github/Sayed-Hossein-Hosseini/SVM_MNIST_Classifier/blob/master/SVM_MNIST_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **SVM MNIST Classifier**

## **Libraries**

In [1]:
import numpy as np
from keras.datasets import mnist
import matplotlib.pyplot as plt
from cvxopt import matrix, solvers
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

## **Load MNIST Data**

In [2]:
# Load MNIST data
(X_train, y_train), (X_test, y_test) = mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


## **Flatten the Images**

In [3]:
# Flatten the images
X_train = X_train.reshape(-1, 28*28) / 255.0
X_test = X_test.reshape(-1, 28*28) / 255.0

## **Hyperparameters**

In [4]:
# Hyperparameters
batch_size = 1000
C = 1.0